In [ ]:
!pip install nnunetv2

In [ ]:
!pip show nnunetv2


In [ ]:
# Cell 1: Imports & Parameters
import os
import subprocess
import time
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Mount Google Drive (if your data is stored there)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# cell 2
os.environ["nnUNet_raw_data_base"] = "/content/drive/MyDrive/nnUNet_raw_data"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_preprocessed"
os.environ["RESULTS_FOLDER"] = "/content/drive/MyDrive/nnUNet_results"

# Also define these additional variables required by nnU-Net:
os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_raw_data"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_results"

# Verify that the directories exist
os.makedirs(os.environ["nnUNet_raw_data_base"], exist_ok=True)
os.makedirs(os.environ["nnUNet_preprocessed"], exist_ok=True)
os.makedirs(os.environ["RESULTS_FOLDER"], exist_ok=True)


In [ ]:
# cell 3
# PARAMETERS
# ============
DATASET_ID = "123"  # Example dataset id; folder name will be Dataset123_Cerebrum7T
DATASET_NAME = f"Dataset{DATASET_ID}_Cerebrum7T"  # This will be "Dataset123_Cerebrum7T"

# Paths for conversion (if needed)
# Update these paths to point to the correct directories in your Google Drive.
# OLD_DATASET_PATH = "/content/drive/MyDrive/datasets/old_dataset"  # Change to your folder path
NNUNET_RAW_FOLDER = "/content/drive/MyDrive/nnUNet_raw_data"

# Paths for inference
TEST_IMAGES_FOLDER = "/content/drive/MyDrive/nnUNet_raw_data/Dataset123_Cerebrum7T/imagesTs"
OUTPUT_PREDICTIONS_FOLDER = "/content/drive/MyDrive/nnUNet_predictions"         # Folder where predictions will be saved

print("Dataset Name:", DATASET_NAME)
# print("Old dataset path:", OLD_DATASET_PATH)
print("nnUNet raw folder:", NNUNET_RAW_FOLDER)
print("Test images folder:", TEST_IMAGES_FOLDER)
print("Output predictions folder:", OUTPUT_PREDICTIONS_FOLDER)


Dataset Name: Dataset123_Cerebrum7T
nnUNet raw folder: /content/drive/MyDrive/nnUNet_raw_data
Test images folder: /content/drive/MyDrive/nnUNet_raw_data/Dataset123_Cerebrum7T/imagesTs
Output predictions folder: /content/drive/MyDrive/nnUNet_predictions


In [ ]:
def plan_and_preprocess():
    command = f"nnUNetv2_plan_and_preprocess -d {DATASET_ID} --verify_dataset_integrity -np 3"
    print("Running experiment planning and preprocessing:")
    print(command)

    # Start the process and combine stdout and stderr
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Loop over each line of output as it is produced
    for line in process.stdout:
        print(line, end='')  # Print each line as it comes

        # Check for key phrases and print additional status messages
        if "Fingerprint extraction..." in line:
            print(">>> Fingerprinting has started.")
        if "Dataset" in line and DATASET_NAME in line:
            print(f">>> Found dataset: {DATASET_NAME}.")
        if "Using" in line and "reader/writer" in line:
            print(">>> Reader/Writer is set.")
        if "2D U-Net configuration:" in line:
            print(">>> 2D U-Net configuration determined.")
        if "3D fullres U-Net configuration:" in line:
            print(">>> 3D fullres U-Net configuration determined.")
        if "3D lowres U-Net configuration:" in line:
            print(">>> 3D lowres U-Net configuration determined.")
        if "Preprocessing dataset" in line:
            print(">>> Preprocessing of the dataset has started.")
        if "Plans were saved to" in line:
            print(">>> Experiment plans saved.")
        if "Preprocessing completed" in line:
            print(">>> Preprocessing is completed.")

    # Wait for the process to finish and check the return code
    process.wait()
    if process.returncode != 0:
        raise Exception("Experiment planning and preprocessing failed.")
    print("Planning and preprocessing completed.")

# Now run the function:
plan_and_preprocess()


Running experiment planning and preprocessing:
nnUNetv2_plan_and_preprocess -d 123 --verify_dataset_integrity -np 3
Fingerprint extraction...
>>> Fingerprinting has started.
Dataset123_Cerebrum7T
>>> Found dataset: Dataset123_Cerebrum7T.
Using <class 'nnunetv2.imageio.nibabel_reader_writer.NibabelIO'> reader/writer
>>> Reader/Writer is set.
Affine image: [[ 6.28471315e-01  3.66661996e-02  2.36148238e-02 -8.86875153e+01]
 [-3.29038538e-02  6.16751671e-01 -9.57958996e-02 -6.18538513e+01]
 [-2.91548371e-02  9.43290293e-02  6.17163241e-01 -1.46295929e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 
Affine seg: [[ 6.28464341e-01  3.66662033e-02  2.36148238e-02 -8.86875153e+01]
 [-3.29034887e-02  6.16751671e-01 -9.57958996e-02 -6.18538513e+01]
 [-2.91545112e-02  9.43290368e-02  6.17163241e-01 -1.46295929e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Image files: ['/content/drive/MyDrive/nnUNet_raw_data/Dataset123_Cerebrum7T/imagesTr/sub-031

In [ ]:
# Cell 4: Model Training
def train_model(configuration, fold):

    command = f"nnUNetv2_train {DATASET_NAME} {configuration} {fold} --npz"
    print(f"Training {configuration} model on fold {fold} with command:")
    print(command)
    subprocess.run(command, shell=True, check=True)
    print(f"Training for {configuration} fold {fold} completed.")

# Train both 3D full resolution and 2D models over 5 folds.
for fold in range(5):
    # Train 3D full resolution model
    train_model("3d_fullres", fold)
    # Train 2D model
    train_model("2d", fold)


In [ ]:
# Cell 5: Inference
def run_inference(configuration):

    command = (
        f"nnUNetv2_predict -i {TEST_IMAGES_FOLDER} -o {OUTPUT_PREDICTIONS_FOLDER} "
        f"-d {DATASET_ID} -c {configuration} --save_probabilities"
    )
    print(f"Running inference for configuration {configuration} with command:")
    print(command)
    subprocess.run(command, shell=True, check=True)
    print(f"Inference for configuration {configuration} completed.")

# Run inference using the 3D full resolution model
run_inference("3d_fullres")


In [ ]:
# Cell 6: Statistical Analysis & Visualization Functions
def load_nifti_image(nifti_path):
    """
    Load a NIfTI image and return its data and affine.
    """
    img = nib.load(nifti_path)
    data = img.get_fdata()
    affine = img.affine
    return data, affine

def visualize_slice(image_data, slice_index=None, title="Slice"):
    """
    Display a single slice from the 3D image data.
    """
    # Choose the middle slice if not provided
    if slice_index is None:
        slice_index = image_data.shape[2] // 2
    plt.figure(figsize=(6, 6))
    plt.imshow(image_data[:, :, slice_index].T, cmap="gray", origin="lower")
    plt.title(f"{title} - Slice {slice_index}")
    plt.axis("off")
    plt.show()

def compute_region_volume(segmentation, label):
    """
    Compute the volume (in voxels) for a given label in the segmentation.
    """
    return np.sum(segmentation == label)


In [ ]:
# Cell 7: Load & Visualize a Predicted Segmentation
predicted_file = os.path.join(output, "sub-001_0000.nii.gz")  # Change as needed

if os.path.exists(predicted_file):
    seg_data, seg_affine = load_nifti_image(predicted_file)
    visualize_slice(seg_data, title="Predicted Segmentation")
else:
    print(f"Predicted file {predicted_file} not found. Please check your inference output folder.")


In [ ]:
# Cell 8: Basic Statistical Analysis Example

ground_truth_file = os.path.join(os.environ["nnUNet_raw_data_base"], "Dataset123_Cerebrum7T", "labelsTr", "sub-001.nii.gz")
# Using the same predicted file from Cell 7
predicted_file = os.path.join(output, "sub-001_0000.nii.gz")

# Check if both files exist
if os.path.exists(ground_truth_file) and os.path.exists(predicted_file):
    gt_data, _ = load_nifti_image(ground_truth_file)
    pred_data, _ = load_nifti_image(predicted_file)

    label = 1
    gt_volume = compute_region_volume(gt_data, label)
    pred_volume = compute_region_volume(pred_data, label)

    print(f"Ground truth volume for label {label}: {gt_volume} voxels")
    print(f"Predicted volume for label {label}: {pred_volume} voxels")
else:
    print("Ground truth or predicted file not found. Please verify the file paths.")
